## Load Open Web UI Token

In [1]:
import yaml
with open('conf.yaml', 'r') as file:
    conf = yaml.safe_load(file)
token = conf["WEB_UI_KEY"]

## Connects to OpenWebUI (RAGaRenn)

In [2]:
from data_boost import WebUIConnector, create_acronym_prompt

url = "https://llmrns.rp.eskn.fr/marius.garenaux-gruau@irisa.fr/api/chat/completions"
owui = WebUIConnector(token, url)

## Creates custom prompt and asks RAGaRenn

In [3]:
import json

n_conv = 10
with open("./data/raw_data.json", "rt") as f:
    all_acro = json.load(f)

boosted_convs = []
n_acros = len(all_acro)
for k, (acro, acro_def) in enumerate(all_acro.items()):
    prompt = create_acronym_prompt(n_conv, acro, acro_def)
    answer = owui.get_chat_response(prompt, return_list=True)
    boosted_convs += answer
    print(k, "on", n_acros)


0 on {'HPC': 'High Performance Computing', 'NumPEx': "Numérique Pour l'Exascale", 'Exa-AToW': 'Architectures and Tools for Large-Scale Workflows'}
1 on {'HPC': 'High Performance Computing', 'NumPEx': "Numérique Pour l'Exascale", 'Exa-AToW': 'Architectures and Tools for Large-Scale Workflows'}
2 on {'HPC': 'High Performance Computing', 'NumPEx': "Numérique Pour l'Exascale", 'Exa-AToW': 'Architectures and Tools for Large-Scale Workflows'}


## Saves dataset of conversations

In [6]:
with open("./data/boosted_data.json", "wt") as f:
    json.dump(boosted_convs, f, indent=4)